In [2]:
import json,os
import boto3
import requests
from requests_aws4auth import AWS4Auth,AWS4SigningKey
from IPython.display import display, Markdown



In [670]:
def printMD(string):
    display(Markdown(string))

In [790]:
def get_awsauth():
    credentials = boto3.Session().get_credentials()
    region = boto3.Session().region_name
    awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, 'lambda', session_token=credentials.token)
    return awsauth

In [671]:
# !pip install requests_aws4auth

In [672]:
# print(awsauth.access_id)  
# print(awsauth.region)        
# print(awsauth.service )     
# print(awsauth.date )       
# print(awsauth.signing_key.scope)  
# print(awsauth.signing_key.key) 
# print(awsauth.signing_key.date)  
# # print(awsauth.get_canonical_headers('host;x-amz-content-sha256;x-amz-date'))

In [4]:
url = 'https://343vvxctf2qepvy5s4dpmje27a0umyuo.lambda-url.us-east-2.on.aws/todos/river'

## OPENAPI tool test

### init ChatGPT

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [6]:
os.environ["OPENAI_API_KEY"] = 'sk-'

In [141]:
llm = ChatOpenAI()

In [53]:
prompt = PromptTemplate(
    input_variables=["input"],
    template="{input}",
)

In [57]:
chain = LLMChain(llm=llm, prompt=prompt,verbose=True )

In [59]:
chain.run({'input':'hello'})



> Entering new LLMChain chain...
Prompt after formatting:
hello

> Finished chain.


'Hello! How can I assist you today?'

## init sagemaker

In [117]:
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain import PromptTemplate, SagemakerEndpoint
from typing import Dict, List


In [813]:
class llmContentHandler(LLMContentHandler):
    parameters = {
        "max_length": 2048,
        "temperature": 0.01,
    }
    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        #ChatGLM参考typescript api描述，进行结构化提取时，有\t会导致失败，需要把\t替换成空格
        input_str = json.dumps({'inputs': prompt,'history':[],**model_kwargs})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["outputs"]

In [814]:
llmcontent_handler = llmContentHandler()
llm_sg=SagemakerEndpoint(
        endpoint_name='chatglm-2023-05-25-06-22-30-689-endpoint', 
        region_name='us-east-2', 
        model_kwargs={'parameters':llmcontent_handler.parameters},
        content_handler=llmcontent_handler
    )

### openapichain

In [676]:
from langchain.tools import OpenAPISpec, APIOperation
from langchain.chains import OpenAPIEndpointChain
from langchain.requests import Requests
from langchain.requests import TextRequestsWrapper


In [678]:
from typing import Any, Dict, Optional

import aiohttp
import requests
from pydantic import BaseModel, Extra

class AWSRequests(Requests):
    """Wrapper around requests to handle auth and async.

    The main purpose of this wrapper is to handle authentication (by saving
    headers) and enable easy async methods on the same base object.
    """

    headers: Optional[Dict[str, str]] = None
    auth: Optional[Any] = None
    # aiosession: Optional[aiohttp.ClientSession] = None

    class Config:
        """Configuration for this pydantic object."""
        extra = Extra.allow
        arbitrary_types_allowed = True
    
    def get(self, url: str, **kwargs: Any) -> requests.Response:
        """GET the URL and return the text."""
        return requests.get(url, headers=self.headers,auth=self.auth, **kwargs)

    def post(self, url: str, data: Dict[str, Any], **kwargs: Any) -> requests.Response:
        """POST to the URL and return the text."""
        return requests.post(url, json=data, headers=self.headers,auth=self.auth,  **kwargs)

    def patch(self, url: str, data: Dict[str, Any], **kwargs: Any) -> requests.Response:
        """PATCH the URL and return the text."""
        return requests.patch(url, json=data, headers=self.headers,auth=self.auth,  **kwargs)

    def put(self, url: str, data: Dict[str, Any], **kwargs: Any) -> requests.Response:
        """PUT the URL and return the text."""
        return requests.put(url, json=data, headers=self.headers,auth=self.auth,  **kwargs)

    def delete(self, url: str, **kwargs: Any) -> requests.Response:
        """DELETE the URL and return the text."""
        return requests.delete(url, headers=self.headers,auth=self.auth,  **kwargs)


## 读取api 文件

In [683]:
openai_api_spec = OpenAPISpec.from_file('api.yaml')

Attempting to load an OpenAPI 3.0.2 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.2 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [684]:
operation = APIOperation.from_openapi_spec(openai_api_spec, '/todos/{username}', "get")
operation.to_typescript()

'/* Get the list of todos */\ntype getTodos = (_: {\n/* The name of the user. */\n\t\tusername: string,\n}) => any;'

## overwrite prompt template

In [804]:
REQUEST_TEMPLATE = """你是一个AI助手。请根据用户指令提取出传递给 agentFunc() 函数的 JSON 参数。

参考API_SCHEMA:
```typescript
{schema}
```

用户指令: "{instructions}"

你的参数必须是以 Markdown 块形式的纯JSON。

例如
参数: ```json
{{符合API_SCHEMA的有效的json}}
```

开始
-----
参数:
"""

RESPONSE_TEMPLATE = """请根据以下API响应，根据用户的指令回答.
API响应: {response}

用户指令: "{instructions}"

你必须根据API响应来回答问题
答案:
---
"""

## 定制output parser

In [805]:
from langchain.schema import BaseOutputParser
import re
class ChatGLMAPIResponderOutputParser(BaseOutputParser):
    """Parse the response and error tags."""


    def parse(self, llm_output: str) -> str:
        """Parse the response and error tags."""
        return llm_output
        # json_match = re.search(r"```json(.*?)```", llm_output, re.DOTALL)
        # if json_match:
        #     return self._load_json_block(json_match.group(1).strip())
        # else:
        #     # raise ValueError(f"No response found in output: {llm_output}.")
        #     return llm_output

    @property
    def _type(self) -> str:
        return "api_responder"
    
class ChatGLMAPIRequesterOutputParser(BaseOutputParser):
    """Parse the request and error tags."""

    def _load_json_block(self, serialized_block: str) -> str:
        try:
            return json.dumps(json.loads(serialized_block, strict=False))
        except json.JSONDecodeError:
            return "ERROR serializing request."

    def parse(self, llm_output: str) -> str:
        """Parse the request and error tags."""
        print(f'llm_output:{llm_output}')

        json_match = re.search(r"```json(.*?)```", llm_output, re.DOTALL)
        if json_match:
            return self._load_json_block(json_match.group(1).strip())
        message_match = re.search(r"```text(.*?)```", llm_output, re.DOTALL)
        if message_match:
            return f"MESSAGE: {message_match.group(1).strip()}"
        return "ERROR making request"

    @property
    def _type(self) -> str:
        return "api_requester"

### 测试get

In [815]:
operation = APIOperation.from_openapi_spec(openai_api_spec, '/todos/{username}', "get")
chain = OpenAPIEndpointChain.from_api_operation(
    operation, 
    llm_sg, 
    requests=AWSRequests(auth=get_awsauth()), 
    verbose=True,
    return_intermediate_steps=True # Return request and response text
)
chain.api_request_chain.prompt.template=REQUEST_TEMPLATE
# chain.api_request_chain.prompt.output_parser = ChatGLMAPIRequesterOutputParser()
chain.api_response_chain.prompt.template=RESPONSE_TEMPLATE
chain.api_response_chain.prompt.output_parser = ChatGLMAPIResponderOutputParser()
answer = chain("列出river的todos")



> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
你是一个AI助手。请根据用户指令提取出传递给 agentFunc() 函数的 JSON 参数。

参考API_SCHEMA:
```typescript
/* Get the list of todos */
type getTodos = (_: {
/* The name of the user. */
		username: string,
}) => any;
```

用户指令: "列出river的todos"

你的参数必须是以 Markdown 块形式的纯JSON。

例如
参数: ```json
{符合API_SCHEMA的有效的json}
```

开始
-----
参数:


> Finished chain.
{"username": "river"}
[{"username": "river", "todos": "[\"item1\", \"item2\", \"item3\", \"item3\", \"eat\", \"eat\", \"eat\", \"item3\", \"item3\", \"\\u56de\\u5bb6\\u5403\\u996d\", \"\\u56de\\u5bb6\\u5403\\u996d2\", \"\\u56de\\u5bb6\\u5403\\u996d3\", \"eat4\", \"sleep\", \"sleep\", \"sleep\"]"}]


> Entering new APIResponderChain chain...
Prompt after formatting:
请根据以下API响应，根据用户的指令回答.
API响应: [{"username": "river", "todos": "[\"item1\", \"item2\", \"item3\", \"item3\", \"eat\", \"eat\", \"eat\", \"item3\", \"item3\", \"\\u56de\\u5bb6\\u5403\\u996d\", \"\\u5

In [816]:
printMD(answer['output'])

根据API响应，可以得出River的todos为：

- \"item1\"
- \"item2\"
- \"item3\"
- \"item3\"
- \"eat\"
- \"eat\"
- \"eat\"
- \"item3\"
- \"item3\"
- \"\\u56de\\u5bb6\\u5403\\u996d\"
- \"\\u56de\\u5bb6\\u5403\\u996d2\"
- \"\\u56de\\u5bb6\\u5403\\u996d3\"
- \"eat4\"
- \"sleep\"
- \"sleep\"

因此，River的todos为上述列表中的所有内容。

In [610]:
my_bytes = '\\u56de\\u5bb6\\u5403\\u996d3'
my_bytes.encode('utf-8').decode('unicode_escape')

'回家吃饭3'

### 测试post

In [726]:
operation = APIOperation.from_openapi_spec(openai_api_spec, '/todos/{username}', "post")
chain = OpenAPIEndpointChain.from_api_operation(
    operation, 
    llm_sg, 
    requests=AWSRequests(auth=get_awsauth()), 
    verbose=True,
    return_intermediate_steps=True # Return request and response text
)
chain.api_request_chain.prompt.template=REQUEST_TEMPLATE
chain.api_response_chain.prompt.template=RESPONSE_TEMPLATE
chain.api_response_chain.prompt.output_parser = ChatGLMAPIResponderOutputParser()
answer = chain("请给river增加一个todo，内容是‘sleep and getup’")



> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
你是一个AI助手。请根据用户指令提取出传递给 agentFunc() 函数的 JSON 参数。

参考API_SCHEMA:
```typescript
/* Add a todo to the list */
type addTodo = (_: {
/* The todo to add to the list. */
  todo: string,
/* The name of the user. */
		username: string,
}) => any;
```

用户指令: "请给river增加一个todo，内容是‘sleep and getup’"

你的参数必须是以 Markdown 块形式的纯JSON。

例如
参数: ```json
{符合API_SCHEMA的有效的json}
```

开始
-----
参数:


> Finished chain.
{"todo": "sleep and getup"}
"add todo list success"


> Entering new APIResponderChain chain...
Prompt after formatting:
请根据以下API响应，根据用户的指令回答.
API响应: "add todo list success"

用户指令: "请给river增加一个todo，内容是‘sleep and getup’"

你必须根据API响应来回答问题
答案:
---


> Finished chain.
给River增加一个todo，内容是“sleep and getup”，成功。

> Finished chain.


## 测试LLM基础能力 

In [862]:
#chatglm

REQUEST_TEMPLATE = """你是一个AI助手。请参考API_SCHEMA:
```typescript
{schema}
```

根据用户指令提取出传递给 agentFunc() 函数的JSON 参数

用户指令: "{instructions}"

你的参数必须是以 Markdown形式的纯JSON。

参数: ```json
{{符合API_SCHEMA的有效的json}}
```

开始
-----
参数:
"""

prompt = PromptTemplate(
    input_variables=["instructions","schema"],
    template=REQUEST_TEMPLATE,
)

api_schema = """/* Add a todo to the list */
type addTodo = (_: {
/* The todo to add to the list. */
  todo: string,
/* The name of the user. */
		username: string,
}) => any;
"""

In [863]:
input_str = "给xiao增加一个todo，内容是‘sleep and sleep’"

### chatglm

In [864]:
chain = LLMChain(llm=llm_sg, prompt=prompt,verbose=False )
resp = chain.run({'instructions':input_str, "schema":api_schema})
printMD(resp)

```json
{
  "todo": "sleep and sleep"
}
```

### chatgpt

In [861]:
chain = LLMChain(llm=llm, prompt=prompt,verbose=False )
resp = chain.run({'instructions':input_str,"schema":api_schema})
printMD(resp)

```json
{"username":"xiao","todo":"sleep and sleep"}
```

## OpenAPI agents

In [147]:
import os, yaml
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec

In [276]:
with open("api.yaml") as f:
    raw_openai_api_spec = yaml.load(f, Loader=yaml.Loader)
openai_api_spec = reduce_openapi_spec(raw_openai_api_spec)

In [277]:
from langchain.agents.agent_toolkits.openapi import planner
from langchain.requests import RequestsWrapper

In [278]:
requests_wrapper = RequestsWrapper(headers={'Content-Type':'application/json'})


todo_agent = planner.create_openapi_agent(openai_api_spec, aws_requests, llm_sg)
user_query = "list all river's todo."
todo_agent.run(user_query)



> Entering new AgentExecutor chain...


KeyboardInterrupt: 